# Task 2

In [272]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import random
from sklearn import preprocessing


# Obtaining and normalizing MNIST dataset helped by: https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627
# Mean and Standart Deviation 0.5 each for data normalization [-1, 1] 
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),

                               ])
# No normalization
transform2 = transforms.ToTensor()


trainset = datasets.MNIST('', download=True, train=True, transform=transform2)
valset = datasets.MNIST('', download=True, train=False, transform=transform2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

dataiter = iter(trainloader)
images, labels_ = dataiter.next()

testiter = iter(valloader)
images1, labels1 = testiter.next()

# Turning the dataset into hardcoded for now 64 size batch of flattened 28*28 pixels
# And one hot encoding for the labels

x_train = images.numpy()
x_train = np.reshape(x_train, (64,28*28))
labels = labels_.numpy()
labels = labels.reshape(-1, 1)
ohe = preprocessing.OneHotEncoder()
ohe.fit(labels)
y_train = ohe.transform(labels).toarray()



x_test = images1.numpy()
x_test = np.reshape(x_test, (64,28*28))
labels1 = labels1.numpy()
labels1 = labels1.reshape(-1, 1)
ohe.fit(labels1)
y_test = ohe.transform(labels1).toarray()



#plt.imshow(images[0].numpy().squeeze(), cmap='gray_r')





losses=[]

# 1 hidden layer, 5 units sigmoid
# Weights initialized in random uniform
class NeuralNetwork:
    def __innit__(self):
        self.learning_rate = 0.001
        self.units = 5
        self.layers = 2
        self.weights_input = np.random.uniform(-1, 1, (self.units, 784))
        self.weights_h1 = np.random.uniform(0, 1, (self.units, self.units))
        self.weights_output = np.random.uniform(0, 1, (10, self.units))
        self.bias_input = np.random.uniform(0,1, (self.units))
        self.bias_h1 = np.random.uniform(0,1, (self.units))
        self.bias_output = np.random.uniform(0,1, (10))
    
    def sigmoid(self, x):
        return 1.0/(1.0+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1-self.sigmoid(x))
    
    def training(self):
        ########################################
        for i in range(64):
        
            sigmoid_activation = self.forward_sigmoid(x_train[i])
            softmax_activation = self.softmax_layer(sigmoid_activation)
            # Mean squared error function
            error = softmax_activation-y_train[i]
            #print("first weights",self.weights_input)
            # Dot product of the softmax derivate matrix and cost vector
            # https://www.python-course.eu/softmax.php
            delta_output =  np.dot(error, self.softmax_derivative(softmax_activation))
            
            #delta_output = np.outer(sigmoid_activation, np.dot(error,self.softmax_derivative(softmax_activation)))
            self.bias_output += error
            
            delta_input = np.dot(delta_output, self.weights_output)
            temp = np.dot(delta_input, self.sigmoid_derivative(sigmoid_activation))
            
            
            self.weights_output += np.outer(sigmoid_activation, delta_output).T * self.learning_rate

            self.weights_input += np.dot(x_train[0], temp) * self.learning_rate
            self.bias_input += delta_input
        
            # Hidden layer
            #hidden_layer_error = np.dot(self.weights_output.T, delta_output)
            #hidden_layer_delta = hidden_layer_error * self.sigmoid_derivative(sigmoid_activation)
        
        
        
            #weights_input = self.sigmoid_derivative(sigmoid_activation) * hidden_layer_error * x_train[0] 
        
            #self.weights_input += self.learning_rate * (self.weights_input.T @ hidden_layer_delta) 
            #self.bias_input += hidden_layer_delta
            print("error",error)
        
            #self.weights_output += self.learning_rate  * (delta_output @ softmax_activation)
            #self.bias_output += delta_output
        
            losses.append(np.sum(np.square(error)))
            #print(np.square(error))
            print("predicted",softmax_activation)
            print("expected",y_train[i])
        
        
        #print("delta output", delta_output)
        #print("hidden layer error", hidden_layer_error)
        #print("hidden layer delta", hidden_layer_delta)
        #print("second weights", self.weights_input)
        print("weights",self.weights_output)

        print("softmax derivative", self.softmax_derivative(softmax_activation))
        #print("sigmoid activation", sigmoid_activation)
        #print("softmax activation", softmax_activation)
        #print("error", error)
        
        
        
    def forward_sigmoid(self, x):
        return self.sigmoid(np.dot(self.weights_input, x))#+self.bias_input)
    def sigmoid_test(self, x):
        return self.sigmoid(np.dot(self.weights_output, x))#+self.bias_input)
    
    def softmax(self, x):
        e = np.exp(x - x.max())
        return e / e.sum()
    def softmax_layer(self, data):
        return self.softmax(np.dot(self.weights_output, data))#+ self.bias_output)
    
    def softmax_derivative(self, x):
        # Taken from https://stackoverflow.com/questions/54976533/derivative-of-softmax-function-in-python
        s = x.reshape(-1,1)
        return np.diagflat(s) - np.dot(s, s.T)


    def SGD(self, epochs, batch_size):
        lmao = 1

    def nupyzdintas(self):
        for i in range(64):
            sigmoid_activation = self.forward_sigmoid(x_train[i])
            softmax_activation = self.sigmoid_test(sigmoid_activation)
            # Mean squared error function
            error = softmax_activation-y_train[i]
        
            delta_output =  np.dot(error, self.sigmoid_derivative(softmax_activation))
            
            #delta_output = np.outer(sigmoid_activation, np.dot(error,self.softmax_derivative(softmax_activation)))
            self.bias_output += error
            
            delta_input = np.dot(delta_output, self.weights_output)
            temp = np.dot(delta_input, self.sigmoid_derivative(sigmoid_activation))
            
            
            self.weights_output += np.outer(sigmoid_activation, delta_output).T * self.learning_rate
            print("delta output", delta_output)
            print("delta input", delta_input)
            print("tmp", temp)
            self.weights_input += np.outer(x_train[0], temp) * self.learning_rate
            self.bias_input += delta_input
       

    


 
    
    

test = NeuralNetwork()
test.__innit__()
test.nupyzdintas()
#test.training()
#test.test()
plt.plot(losses)
plt.show




#sigmoid layer
#epoch (means passing the entire data set. One epoch contains number_of_items/batch_size iterations)
#forward pass (refers to calculation process, values of the output layers from the inputs data. It's traversing through all neurons from first to last layer.)
#backward pass (refers to process of counting changes in weights (de facto learning), using gradient descent algorithm (or similar). Computation is made from last layer, backward to the first layer.)



#Neural Networks
#Parameterized number and types of layers. number of units

delta output 1.5106110122690795
delta input [[1.02307009 0.35946786 1.14818439 0.35614362 1.31145313]
 [0.54436708 0.55136317 0.11142597 0.58024604 0.58790094]
 [0.26334682 0.79188457 0.33616118 0.18953968 1.35900631]
 [1.50551877 0.08229742 0.14958062 0.07948892 0.69799874]
 [0.54143038 0.42470667 0.3060362  1.31617876 0.11467703]
 [0.43385464 0.45582013 0.52720594 0.49734856 0.76017713]
 [0.74376981 0.98900474 0.68896893 1.37971687 1.11821505]
 [1.29386188 0.22761481 0.70497497 0.48000246 0.3902462 ]
 [0.33391745 0.32593675 1.4150393  0.53844458 1.0689458 ]
 [0.91582742 1.43010305 1.29512151 1.35810679 1.32124057]]
tmp [0.89915899 0.52624919 0.63470596 0.57757636 0.58545711 0.57411715
 1.06229931 0.68970902 0.76046139 1.37031425]


ValueError: operands could not be broadcast together with shapes (5,784) (784,10) (5,784) 

In [251]:
def sigmoidLayer(units, x):
    weights
    

In [40]:
sizes = [2, 3, 1]
biases = [np.random.randn(y, 1) for y in sizes[1:]]
print(biases)

weights = np.random.normal(size=400)
print(weights)

[array([[-0.53619004],
       [ 1.56651314],
       [ 1.80161131]]), array([[0.56628081]])]
[ 9.60957939e-01  2.03613961e-01 -8.01874742e-01 -5.65171013e-01
  1.41022351e+00 -1.95997209e+00 -1.66705420e-02  1.12882664e+00
 -3.49296312e-01 -1.54713477e+00  5.17084054e-01 -1.45649878e+00
 -7.32995819e-01 -4.55096516e-01  8.91261789e-01  2.14952221e+00
 -1.59823079e+00 -9.64555671e-02 -1.24804751e+00 -9.84556013e-01
  3.20040197e-02 -1.30015600e+00  2.42689258e-01  6.90503425e-01
  1.13433117e+00 -2.73671486e-01  6.39863072e-01 -7.16442635e-01
 -1.33345848e-01  3.31146707e-01 -8.13844892e-01  5.59047108e-01
 -8.31024119e-01  8.80314984e-01 -3.48237999e-01 -2.56118282e-01
  9.25833186e-01  2.75111702e-02  2.12381498e-01 -6.99500034e-02
  1.04341481e-01  2.41754096e-01  4.78163883e-01  4.78288056e-02
 -1.76387881e-01  1.52010685e+00 -9.13495203e-01 -1.43378653e+00
  1.12415405e+00 -8.33291985e-01 -9.43379121e-01  4.20352415e-01
  8.43742956e-01 -6.12304078e-01 -3.06407495e-01  1.58556496e+0

In [27]:
# Each row is a training example, each column is a feature  [X1, X2, X3]
X=np.array(([0,0,1],[0,1,1],[1,0,1],[1,1,1]), dtype=float)
y=np.array(([0],[1],[1],[0]), dtype=float)

# Define useful functions    

# Activation function
def sigmoid(t):
    return 1/(1+np.exp(-t))

# Derivative of sigmoid
def sigmoid_derivative(p):
    return p * (1 - p)

# Class definition
class NeuralNetwork:
    def __init__(self, x,y):
        self.input = x
        self.weights1= np.random.rand(self.input.shape[1],4) # considering we have 4 nodes in the hidden layer
        self.weights2 = np.random.rand(4,1)
        self.y = y
        self.output = np. zeros(y.shape)
        
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.layer2 = sigmoid(np.dot(self.layer1, self.weights2))
        return self.layer2
        
    def backprop(self):
        d_weights2 = np.dot(self.layer1.T, 2*(self.y -self.output)*sigmoid_derivative(self.output))
        d_weights1 = np.dot(self.input.T, np.dot(2*(self.y -self.output)*sigmoid_derivative(self.output), self.weights2.T)*sigmoid_derivative(self.layer1))
    
        self.weights1 += d_weights1
        self.weights2 += d_weights2

    def train(self, X, y):
        self.output = self.feedforward()
        self.backprop()
        

NN = NeuralNetwork(X,y)
for i in range(1500): # trains the NN 1,000 times
    if i % 100 ==0: 
        print ("for iteration # " + str(i) + "\n")
        print ("Input : \n" + str(X))
        print ("Actual Output: \n" + str(y))
        print ("Predicted Output: \n" + str(NN.feedforward()))
        print ("Loss: \n" + str(np.mean(np.square(y - NN.feedforward())))) # mean sum squared loss
        print ("\n")
  
    NN.train(X, y)

for iteration # 0

Input : 
[[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]
Actual Output: 
[[0.]
 [1.]
 [1.]
 [0.]]
Predicted Output: 
[[0.84365544]
 [0.89140871]
 [0.87749095]
 [0.90791675]]
Loss: 
0.39071696695047575


for iteration # 100

Input : 
[[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]
Actual Output: 
[[0.]
 [1.]
 [1.]
 [0.]]
Predicted Output: 
[[0.47487267]
 [0.49405257]
 [0.53785013]
 [0.52405324]]
Loss: 
0.24242528626364135


for iteration # 200

Input : 
[[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]
Actual Output: 
[[0.]
 [1.]
 [1.]
 [0.]]
Predicted Output: 
[[0.19598313]
 [0.60960974]
 [0.76024414]
 [0.41812489]]
Loss: 
0.10578131045642392


for iteration # 300

Input : 
[[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]
Actual Output: 
[[0.]
 [1.]
 [1.]
 [0.]]
Predicted Output: 
[[0.06807624]
 [0.86772868]
 [0.88472276]
 [0.15219759]]
Loss: 
0.014645756322074838


for iteration # 400

Input : 
[[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 1.]]
Actual Output: 
[[0.]
